In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers. imoprt Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import CategoricalAccuracy,SparseCategoricalAccuracy,SparseTopCategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.layers.experimental import preprocessing
import time
import pandas as pd
from shutil import copy2

In [ ]:
train_directory = '...'
validation_directory = '...'
world_files = '...'
dst = '...'
INPUT_DIM = ...

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    image_size = (INPUT_DIM, INPUT_DIM),
    batch_size = 4096)

In [ ]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_directory,
    image_size = (INPUT_DIM, INPUT_DIM),
    batch_size = 4096)


In [ ]:
base_model=tf.keras.applications.VGG16(input_shape=(224,224,3),
                                      include_top=True,
                                      weights='imagenet')
model=base_model
model.summary()

In [ ]:
feature_maps=[layer.output for layer in model.layers[:5]]
activation_model=Model(inputs=model.input,outputs=feature_maps)
activation_model.summary()

In [ ]:
test-image='...'
image=tf.keras.preprocessing.image.load_img(test_imag,color_mode='rgb',target_size=(224,224))
image=tf.convert_to_tensor(tf.keras.preprocessing.image.img_to_array(image))/255.
activations=activation_model.predict(tf.expand_dims(image,0))

In [ ]:
for i in range(5):
    print(activations[i].shape)

In [ ]:
image_size=5
num_of_channels=4
for i in range(1,len(activations)):
    print("Feature map number ---> ",i)
    axes=[]
    fig=plt.figure()
    fig.set_figheight(image_size)
    fig.set_figwidth(image_size)
    
    for channel in range(num_of_channels):
        axes.append(fig.add_subplot(int(np.sqrt(num_of_channels)),int(np.sqrt(num_of_channels)),channel+1))
        subplot_title=("Channel"+str(channel))
        axes[-1].set_title(subplot_title)
        plt.imshow(activations[i][0,:,:,channel])
    fig.tight_layout()
    plt.show()

In [ ]:
backbone=tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                      include_top=True,
                                      weights='imagenet')

backbone.summary()

In [ ]:
backbone.trainable=False

In [ ]:
inputs=Input(shape=(64,64,3))

x=tf.keras.preprocesssing.Rescaling(1/255.)(inputs)
x=backbone(inputs,training=False)
x=GlobalAveragePooling2D(x)
x=Dense(1000)(x)
outputs=Dense(200,activation='softmax')(x)
model=Model(inputs,outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(lr=1e-3,
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseCategoricalAccuracy(),SparseCategoricalAccuracy(k=5)])

In [ ]:
NUM_EPOCHS = 15
history = model.fit(train_dataset, validation_data=val_dataset, epochs=NUM_EPOCHS)

In [ ]:

plt.plot(history.history['loss'][0:60])
plt.plot(history.history['val_loss'][0:60])

plt.title('model_losss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()